In [1]:
import pandas as pd
import sys

import tensorflow as tf
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import pickle
from os import path
from keras.callbacks import ModelCheckpoint

import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
RANDOM_SEED=2245
from numpy.random import seed
seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

import matplotlib.pyplot as plt

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
    

In [3]:
PATH_SCRIPT = "/content/drive/MyDrive/Audio classification/Notebooks-Mahim"
os.chdir(PATH_SCRIPT)
print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1DGiPjXTUI4ejRejy9C8oe3nAQXI6AZIW/Audio classification/Notebooks-Mahim


## For trying out different architectures, I am fitting with validation set, since X_train is very large, and subset is not available at the moment.

In [4]:
# X_train_file = "/content/drive/MyDrive/Audio classification/mfccs/train_13_2048_512.pkl"
# Y_train_file = "/content/drive/MyDrive/Audio classification/lables/train.pkl"

X_train_file = "/content/drive/MyDrive/Audio classification/mfccs/val_13_2048_512.pkl"
Y_train_file = "/content/drive/MyDrive/Audio classification/lables/val.pkl"

with open(X_train_file, mode='rb') as fin:
    # X_train = pickle.load(fin)
    X = pickle.load(fin)

with open(Y_train_file, mode='rb') as fin:
    # Y_train = pickle.load(fin)
    Y = pickle.load(fin)

print(X.shape, Y.shape)

(7711, 1292, 13) (7711, 5)


In [5]:
# del X_train, Y_train

## For now helper_script is kept in a single cell.
## Will be used separately in Lab's Machines

In [6]:
#@title Helper Script is kept here verbatim. Please Update from GitHub regularly !!

from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import seaborn as sns

RANDOM_STATE = 2245

LABELS_MAPPING_FORWARD = {
    'national': 0,
    'international': 1,
    'economics': 2,
    'entertainment': 3,
    'sports': 4
}

LABELS_MAPPING_REVERSE = {
    0: 'national', 
    1: 'international', 
    2: 'economics', 
    3: 'entertainment', 
    4: 'sports'
}

NUM_LABELS = len(LABELS_MAPPING_FORWARD)

get_label_name = lambda label_id: LABELS_MAPPING_REVERSE.get(label_id)
get_label_class_id = lambda name: LABELS_MAPPING_REVERSE.get(name)

def get_various_metrics_and_print(Y_true, Y_predicted):
    TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    accuracy = (TP + TN)/(TP+TN+FP+FN)
    recall = (TP)/(TP + FN)
    specificity = (TN)/(TN + FP) # TNR
    false_positive_rate = (FP)/(TN + FP) # false_positive_rate = 1 - TNR
    precision = (TP)/(TP + FP)
    false_discovery_rate = (FP)/(TP + FP)
    neg_predicted_val = (TN)/(TN + FN)
    f1_score = 2*((precision * recall) / (precision + recall))

    print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
    print("Accuracy = ", accuracy*100, "%")
    print("TPR = Sensitivity = Recall = ", recall*100, "%")
    print("TNR = Specificity = ", specificity*100, "%")
    print("Precision = PPV = Positive Predictive Value = ", precision*100, "%")
    print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
    print("FPR = False Positive Rate = ", false_positive_rate*100, "%")
    print("F1 Score = ", f1_score*100, "%")
    print("Neg Predicted Val = ", neg_predicted_val*100, "%")
    print("\n")
    # c_report = classification_report(y_true=Y_true, y_pred=Y_predicted)
    c_report = None
    print(c_report)
    return precision, recall, f1_score, c_report


def plot_confusion_matrix(Y_true=None, Y_predicted=None):
  ## https://www.youtube.com/watch?v=T27WVIM8Xys
    # mat = confusion_matrix(y_test,y_preds)
    cf_matrix = confusion_matrix(Y_true, Y_predicted)
    # cf_matrix = np.array([[23, 5], [3, 30]])
    print(cf_matrix)
    
    # group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_names = ['TN','FP','FN','TP']
    
    group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
    
    group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
    
    # labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]

    labels = np.asarray(labels).reshape(2,2)
    
    # sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
    #         fmt='.2%', cmap=plt.cm.magma)

    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap=plt.cm.magma)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_STATE)

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(6168, 1292, 13) (6168, 5)
(1543, 1292, 13) (1543, 5)


### Transformer For Audio Classification codes taken from the following GitHub [repository](https://github.com/facundodeza/transfomer-audio-classification/blob/master/audio_classification_transformer.ipynb)

In [8]:
#@title Full Transformer Code from above GitHub repository
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)


class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

    
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('projection layer is linear')
  
  else:
    projection=tf.identity(inputs) # Same layer repeat ?
    print('projection layer is none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)
  
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size, use_bias=True, activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [9]:
print(f"NUM_LABELS = {NUM_LABELS}")

NUM_LABELS = 5


## TODO - Changes

* Custom Learning Rate Scheduler of Transformer
* No. of heads
* No. of layers
* Projection - Different features eg. Linear, None, Convolutional, etc.
* **Beam Search** ?
* Minimum Risk Training later on best model to be optimized on **f1** score



In [10]:
NUM_LAYERS = 2
D_MODEL = X.shape[2] # eg. X.shape = (7711, 1292, 13) -> D_MODEL = 13
NUM_HEADS = 1 # 4
UNITS = 1024
DROPOUT = 0.1
TIME_STEPS = X.shape[1] # TIME_STEPS -> 1292
OUTPUT_SIZE = NUM_LABELS
EPOCHS = 10
# EXPERIMENTS=10
projection = 'none'

In [11]:
print(f"d_model = {D_MODEL}, num_heads = {NUM_HEADS}")

d_model = 13, num_heads = 1


In [17]:
CKPT_DIRECTORY = "CKPTS-June-25"
checkpoint = ModelCheckpoint(CKPT_DIRECTORY, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max') ## need to monitor val_get_f1 ?
callbacks_list = [checkpoint]

model = transformer(
    time_steps=TIME_STEPS,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    output_size=OUTPUT_SIZE,  
    projection=projection
)

model.compile(optimizer=tf.keras.optimizers.Adam(0.000001), ## Need to implement custom LR Scheduler
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

projection layer is none


In [18]:
print(model.summary())

Model: "audio_class"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None, 13)]   0                                            
__________________________________________________________________________________________________
tf.math.reduce_sum_2 (TFOpLambd (None, None)         0           inputs[0][0]                     
__________________________________________________________________________________________________
tf.cast_2 (TFOpLambda)          (None, None)         0           tf.math.reduce_sum_2[0][0]       
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           tf.cast_2[0][0]                  
________________________________________________________________________________________

In [ ]:
history = model.fit(X_train, Y_train, 
                epochs=EPOCHS, 
                validation_data=(X_test, Y_test),
                callbacks = callbacks_list, verbose=1)

# model.save(MODEL_SAVE_FILE)

Epoch 1/10
193/193 [==============================] - ETA: 0s - loss: 1.8940 - accuracy: 0.2579

In [ ]:
Y_pred_0 = model.predict(X_test[0:1])
print(Y_pred_0.shape)
Y_pred_0 = Y_pred_0[0]
print(Y_pred_0)
print(np.argmax(Y_pred_0))


In [ ]:
plt.plot(history)
plt.xtitle("")
plt.ytitle("")

In [ ]:
Y_pred_test = model.predict(X_test)

In [ ]:
# get_various_metrics_and_print(Y_true=Y_test, Y_predicted=Y_pred_test)

In [ ]:
# with open("Y_pred_test.pkl", mode='wb') as fout:
#     pickle.dump(Y_pred_test, fout)

# with open("Y_test.pkl", mode='wb') as fout:
#     pickle.dump(Y_test, fout)

In [ ]:
Y_test_labeled = np.argmax(Y_test, axis=-1)
Y_test_labeled = [get_label_name(id) for id in Y_test_labeled]
print(Y_test_labeled)

Y_pred_labeled = np.argmax(Y_pred_test, axis=-1)
Y_pred_labeled = [get_label_name(id) for id in Y_pred_labeled]
print(Y_pred_labeled)

In [ ]:
c_report = classification_report(y_true=Y_test_labeled, y_pred=Y_pred_labeled)
print(c_report)

In [ ]:
# confusion_matrix(y_true=Y_test_labeled, y_pred=Y_pred_labeled)

# plot_confusion_matrix(Y_true=Y_test_labeled, Y_predicted=Y_pred_labeled)

In [ ]:
np.unique(np.argmax(Y, axis=-1), return_counts=True)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()